In [1]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from data_analysis import calculate_metrics, load_weights_and_evaluate
from model_builders import GCN_pretraining
from hyperparameter_tuning_GCN import objective
from functools import partial
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle
import dill
from hyper_mining import objective_fn
import keras.backend as K

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

In [2]:
fspace = {
    'conv1' : hp.quniform('conv1', 32, 64, 8),
    'conv2' : hp.quniform('conv2', 64, 128, 8),
    'conv3' : hp.quniform('conv3', 128, 168, 8),
    'fp' : hp.quniform('fp', 96, 196, 8),
    'dense1' : hp.quniform('dense1',96,512,32),
    'dense2' : hp.quniform('dense2',96,512,32),
    'dense3' : hp.quniform('dense3',64,512,32),
    'dropout_rate' : hp.uniform('dropout_rate',0.1,0.5),
    'lr' : hp.uniform('lr',0.00001,0.01),
    'n_epochs' : hp.quniform('n_epochs',15,60,5),
    'batch_size' : hp.quniform('batch_size',64,256,16),
    'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.1, 1), 
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.1, 1), 
    'gamma' : hp.uniform('gamma', 0.1, 1), 
    'learning_rate' : hp.uniform('learning_rate', 0.1, 1),
    'max_delta_step' : hp.quniform('max_delta_step',1,10,1),
    'max_depth' : hp.quniform('max_depth',6, 12, 1),
    'min_child_weight' : hp.quniform('min_child_weight',10 ,500 ,5),
    'reg_alpha' : hp.uniform('reg_alpha',0.1,100),
    'reg_lambda' : hp.uniform('reg_lambda',0.1,100),
    'subsample' : hp.uniform('subsample',0.1,1.0),
    'max_bin' : hp.quniform('max_bin',16,256,16),
    'margin' : hp.uniform('margin',0.2,2)
}

In [3]:
target = 'p38'
base_path = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath = base_path+f'/data/{target}/data.csv'
df=pd.read_csv(data_fpath).set_index('biolab_index')

with open(base_path+f'/data/{target}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds = dill.load(in_f)
with open(base_path+f'/data/{target}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds = dill.load(in_f)

In [4]:
training_list = [df.loc[train_val_folds[0][0]],
                 df.loc[train_val_folds[1][0]],
                 df.loc[train_val_folds[2][0]],
                 df.loc[train_val_folds[3][0]],
                 df.loc[train_val_folds[4][0]],
                 df.loc[train_val_folds[5][0]],
                 ]
validation_list = [df.loc[train_val_folds[0][1]],
                   df.loc[train_val_folds[1][1]],
                   df.loc[train_val_folds[2][1]],
                   df.loc[train_val_folds[3][1]],
                   df.loc[train_val_folds[4][1]],
                   df.loc[train_val_folds[5][1]],
                   ]

In [5]:
fmin_objective = partial(objective, train_sets = training_list, val_sets = validation_list)

In [6]:
def run_trials():

    trials_step = 1  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 2  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("gcn_xgb.hyperopt", "rb"))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn = fmin_objective, space = fspace, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    print("Best:", best)
    
    # save the trials object
    with open("gcn_xgb.hyperopt", "wb") as f:
        pickle.dump(trials, f)
    return(trials)

In [7]:
trials = run_trials()

Found saved Trials! Loading...
Rerunning from 2 trials to 3 (+1) trials
LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
 67%|█████████████████████████████████████████████▎                      | 2/3 [00:05<00:02,  2.72s/trial, best loss=?]

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                 

In [8]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [11]:
trials.trials

[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': -0.880932711114237, 'status': 'ok'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'colsample_bylevel': [0],
    'colsample_bytree': [0],
    'conv1': [0],
    'conv2': [0],
    'conv3': [0],
    'dense1': [0],
    'dense2': [0],
    'dense3': [0],
    'dropout_rate': [0],
    'fp': [0],
    'gamma': [0],
    'learning_rate': [0],
    'lr': [0],
    'margin': [0],
    'max_bin': [0],
    'max_delta_step': [0],
    'max_depth': [0],
    'min_child_weight': [0],
    'n_epochs': [0],
    'reg_alpha': [0],
    'reg_lambda': [0],
    'subsample': [0]},
   'vals': {'colsample_bylevel': [0.7123342985374396],
    'colsample_bytree': [0.9455012808813511],
    'conv1': [48.0],
    'conv2': [80.0],
    'conv3': [152.0],
    'dense1': [256.0],
    'dense2': [448.0],
    'dense3': [160.0],
    'dropout_rate': [0.3560048819669964],
    'fp': [144.0],
    'gamma': [0.3014935659014

In [8]:
best_loss = trials.trials[0]['result']['loss']
for i in range(1,len(trials.trials)):
    if (trials.trials[i]['result']['loss'] <=  best_loss):
        best_loss = trials.trials[i]['result']['loss']
        index = i
best_params = trials.trials[index]['misc']['vals']

NameError: name 'index' is not defined

In [ ]:
from hyper_mining import XGB_predictor,GCN_online_mining
from data_analysis import calculate_metrics
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)
gcn_params = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [int(best_params['conv1'][0]), int(best_params['conv2'][0]), int(best_params['conv3'][0])],
        "fp_length" : [int(best_params['fp'][0]), int(best_params['fp'][0]), int(best_params['fp'][0])],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [int(best_params['dense1'][0]), int(best_params['dense2'][0]), int(best_params['dense3'][0])],
        'dropout_rate' : [best_params['dropout_rate'][0], best_params['dropout_rate'][0]],
        'lr' : best_params['lr'][0],
        'batch_size' : int(best_params['batch_size'][0]),
        'n_epochs' : int(best_params['n_epochs'][0]),
        'margin' : best_params['margin'][0]
        }
xgb_params = {
        "colsample_bylevel" : best_params['colsample_bylevel'][0],
        "colsample_bytree" : best_params['colsample_bytree'][0],
        "gamma" : best_params['gamma'][0],
        "eta" : best_params['learning_rate'][0],
        "max_delta_step" : int(best_params['max_delta_step'][0]),
        "max_depth" : int(best_params['max_depth'][0]),
        "min_child_weight" : int(best_params['min_child_weight'][0]),
        "alpha" : best_params['reg_alpha'][0],
        "lambda" : best_params['reg_lambda'][0],
        "subsample" : best_params['subsample'][0],
        "max_bin" : int(best_params['max_bin'][0]),
        "eval_metric":'auc',
        "objective":'binary:logistic',
        "booster":'gbtree',
        "single_precision_histogram" : True
        }
class_XGB = XGB_predictor(xgb_params)
class_GCN = GCN_online_mining(gcn_params)

In [ ]:
train_for_test = train_test_folds[0] 
val_for_test = train_test_folds[1]

In [ ]:
validation_list[0]

In [ ]:
class_GCN.dataframe_to_gcn_input(training_list[0])

In [ ]:
#K.clear_session()
training_metrics = {}
validation_metrics = {}
for i in range(len(training_list)):
        X_atoms_cold,X_bonds_cold,X_edges_cold = class_GCN.dataframe_to_gcn_input(validation_list[i])
        Y_cold = val_sets[i].Binary 
        Y_dummy_cold = np.empty((X_atoms_cold.shape[0],gcn_params['dense_size'][2]+1))
        X_atoms_train, X_bonds_train, X_edges_train = class_GCN.dataframe_to_gcn_input(training_list[i])
        Y = train_sets[i].Binary
        Y_dummy_train = np.empty((X_atoms_train.shape[0],gcn_params['dense_size'][2]+1))
        
        gcn_encoder = class_GCN.build_encoder()
        gcn_model = class_GCN.build_model(gcn_encoder)
        gcn_mining = class_GCN.build_mining(gcn_model)
        
        gcn_mining.fit([X_atoms_train,X_bonds_train,X_edges_train,Y],
                       Y_dummy_train,
                       epochs = gcn_params['epochs'],
                       batch_size = gcn_params['batch_size'],
                       shuffle = True,
                       validation_data = ([X_atoms_cold,X_bonds_cold,X_edges_cold,Y_cold],Y_dummy_cold)
                      )
        #Predict Embeddings
        embeddings_cold = gcn_model.predict([X_atoms_cold,X_bonds_cold,X_edges_cold])
        embeddings_train = gcn_model.predict([X_atoms_train, X_bonds_train, X_edges_train])
        
        #Prepare data for XGBoost
        dmatrix_train = class_XGB.to_xgb_input(Y,embeddings_train)
        dmatrix_cold = class_XGB.to_xgb_input(Y_cold,embeddings_cold)
        
        evalist = [(dmatrix_train,'train'),(dmatrix_cold,'eval')]
        xgb_model = class_XGB.build_model(dmatrix_train,evalist,300)
        
        xgb_pred_cold = xgb_model.predict(dmatrix_cold)
        validation_metrics['Val_%s'%i] = calculate_metrics(np.array(Y_cold),xgb_pred_cold)
        
        xgb_pred_train = xgb_model.predict(dmatrix_train)
        training_metrics['Train_%s'%i] = calculate_metrics(np.array(Y,xgb_pred_train))
        
